## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%pip install pyspark

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE
# Imports
import numpy as np
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.ml import *
import mlflow
import mlflow.pyfunc
from mlflow.pyfunc import spark_udf
from pyspark.sql.functions import col, when
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from pyspark.sql.functions import col, explode, split, regexp_replace, input_file_name, current_timestamp, to_timestamp

In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

In [0]:
# Shuffle partitions
# spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)
spark.conf.set("spark.sql.shuffle.partitions", 32)
spark.conf.get("spark.sql.shuffle.partitions")

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Setting up the Spark session with Delta support
spark = (SparkSession.builder
         .appName("ReadFileContents")
         .config("spark.sql.adaptive.enabled", "true")
         .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
         .getOrCreate()
)

# Reading the number of source files
source_file_directory = dbutils.fs.ls(TWEET_SOURCE_PATH)
file_count = len(source_file_directory)
print(f"Number of source files: {file_count}")

In [0]:
# Reading the contents of one of the files
sample_file_path = source_file_directory[0].path

# Reading the file into a DataFrame
sample_file_df = spark.read.json(sample_file_path)

# Showing the contents of the DataFrame
display(sample_file_df)

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Define the schema for the raw data
raw_data_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("text", StringType(), True)
])

In [0]:
# Setup Spark to allow schema evolution on the delta table
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

# # Clear previous runs if necessary
# dbutils.fs.rm(BRONZE_CHECKPOINT, True)
# dbutils.fs.rm(BRONZE_DELTA, True)

In [0]:
# Set up a read stream using cloudFiles limited to 100 samples for testing
df_stream = (spark
             .readStream
             .format("cloudFiles")
             .option("cloudFiles.format", "json")
             .option("cloudFiles.maxFilesPerTrigger", 1)
             .schema(raw_data_schema)
             .load(TWEET_SOURCE_PATH)
             .withColumn("source_file", input_file_name())
             .withColumn("processing_time", current_timestamp())
)

In [0]:
# Accelerate queries with Delta: This query is on a Delta table with many small files. To improve the performance of queries, run the OPTIMIZE command on the table delta.`dbfs:/tmp/labuser104917-3007331/bronze.delta`. Learn more

In [0]:
# Setup the write stream to the bronze Delta table
bronze_stream = (df_stream
                 .writeStream
                 .format("delta")
                 .outputMode("append")
                 .option("checkpointLocation", BRONZE_CHECKPOINT)
                 .option("cloudFiles.format", "json")
                 .option("mergeSchema", "true")
                 .option("cloudFiles.schemaLocation", BRONZE_DELTA)
                 .trigger(availableNow=True)
                 .start(BRONZE_DELTA)
)

# Optimize the Delta table to improve query performance
# spark.sql(f"OPTIMIZE delta.`{BRONZE_DELTA}`")

In [0]:
# bronze_stream.awaitTermination()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
display(bronze_stream.status)

In [0]:
# Read the Bronze Delta Table
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# Count the number of tweets
tweet_count = bronze_df.count()

# Display the count
print(f"Number of tweets captured in the Bronze Table: {tweet_count}")

In [0]:
# Display the first 25 values of bronze_df
display(bronze_df.limit(25))

In [0]:
# Adjusted code to handle timestamp type for the 'processing_time' column
nan_null_counts = bronze_df.select(
    [count(when(isnan(c) | isnull(c), c)).alias(c) if bronze_df.schema[c].dataType not in [TimestampType()] else count(when(isnull(c), c)).alias(c) for c in bronze_df.columns]
)

# Display the counts
display(nan_null_counts)

###VERDICT: No NaN or Null Values

In [0]:
# Count the number of tweets by each user handle
user_tweet_counts = bronze_df.groupBy("user").count()

# Sort the data by count in descending order
sorted_user_tweet_counts = user_tweet_counts.orderBy(col("count").desc())

# Display the sorted counts
display(sorted_user_tweet_counts)

In [0]:
# Counting tweets with at least one mention and without any mentions
count_with_mentions = bronze_df.filter(expr("text LIKE '%@%'")).count()
count_without_mentions = bronze_df.filter(expr("text NOT LIKE '%@%'")).count()

# Displaying the counts
print(f"Number of tweets with at least one mention: {count_with_mentions}")
print(f"Number of tweets with no mentions: {count_without_mentions}")

In [0]:
# 5) Plot a bar chart that shows the top 20 tweeters (users)

# Convert to Pandas DataFrame
sorted_user_tweet_counts_pd = sorted_user_tweet_counts.limit(20).toPandas()

# Plot the bar chart
bar_chart_tweeters = px.bar(sorted_user_tweet_counts_pd, 
             x = 'user', y = 'count', 
             title = 'Top 20 Tweeters', 
             labels = {'user': 'User', 'count': 'Number of Tweets'}, 
             height = 600)

bar_chart_tweeters.update_layout(showlegend = True)

# Show the plot
display(bar_chart_tweeters)

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Clear previous runs if necessary
# dbutils.fs.rm(SILVER_CHECKPOINT, True)
# dbutils.fs.rm(SILVER_DELTA, True)

In [0]:
# Setup a read stream on the Bronze Delta Table
bronze_stream = (spark
    .readStream
    .format("delta")
    .load(BRONZE_DELTA)
)

In [0]:
# Define the transformation
silver_stream = (bronze_stream
    .withColumn("timestamp", to_timestamp("date", "EEE MMM dd HH:mm:ss z yyyy")) \
    .withColumn("mention", explode(split(col("text"), " ")))  # Explode mentions into separate rows
    .filter(col("mention").startswith("@"))  # Filtering rows to keep only mentions
    .withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", ""))  # Removing mentions from text
    .select(
        "timestamp", 
        "mention", 
        "cleaned_text", 
        "sentiment", 
        input_file_name().alias("source_file"), 
        current_timestamp().alias("processing_time")
    )
)

In [0]:
# Setup a write stream to append to the Silver Delta Table
silver_stream.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", SILVER_CHECKPOINT) \
    .start(SILVER_DELTA)

In [0]:
silver_stream.awaitTermination()

In [0]:
# Read the Bronze Delta Table
silver_df = spark.read.format("delta").load(SILVER_DELTA)
display(silver_df)

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Import the necessary MLflow module and PySpark functions
# from mlflow.pyfunc import spark_udf
# from pyspark.sql.functions import col, when

# Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
MODEL_NAME = spark_udf(spark, "models:/HF_TWEET_SENTIMENT/Production")

In [0]:
# Set up a streaming read on the silver Delta table
silver_stream = (
    spark.readStream
    .format("delta")
    .option("ignoreChanges", "true")
    .load(SILVER_DELTA)
)

In [0]:
# ENTER YOUR CODE HERE
# Use a spark UDF to parallelize the inference across your silver data
gold_df = silver_stream.withColumn("predictions", MODEL_NAME(col("cleaned_text"))) \
    .select(
        col("timestamp"),
        col("mention"),
        col("cleaned_text"),
        col("sentiment"),
        col("predictions.label").alias("predicted_sentiment"),
        col("predictions.score").alias("predicted_score"),
        when(col("sentiment") == "negative", 0).otherwise(1).alias("sentiment_id"),
        when(col("predictions.label") == "NEGATIVE", 0).otherwise(1)
        .alias("predicted_sentiment_id")
    )

# Setup a write stream to append to the gold delta table
gold_stream = gold_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", GOLD_CHECKPOINT) \
    .start(GOLD_DELTA)

In [0]:
gold_stream.awaitTermination()

In [0]:
# Read the Gold Delta Table
gold_table = spark.read.format("delta").load(GOLD_DELTA)
display(gold_table)

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE

In [0]:
# Convert to Pandas DataFrame for evaluation
gold_pd = gold_table.select("sentiment_id", "predicted_sentiment_id").toPandas()
display(gold_pd)

In [0]:
# Compute metrics
precision = precision_score(gold_pd["sentiment_id"], gold_pd["predicted_sentiment_id"])
recall = recall_score(gold_pd["sentiment_id"], gold_pd["predicted_sentiment_id"])
f1 = f1_score(gold_pd["sentiment_id"], gold_pd["predicted_sentiment_id"])

# Compute confusion matrix
conf_matrix = confusion_matrix(gold_pd["sentiment_id"], gold_pd["predicted_sentiment_id"])

In [0]:
%pip install -U kaleido

In [0]:
# Plot confusion matrix using plotly.express
fig = px.imshow(
    conf_matrix,
    labels=dict(x="Predicted", y="Actual", color="Count"),
    x=["Negative", "Positive"],
    y=["Negative", "Positive"],
    title="Confusion Matrix"
)

# Save confusion matrix plot
conf_matrix_path = "/tmp/confusion_matrix.png"
fig.write_image(conf_matrix_path)

In [0]:
# Start MLflow run
with mlflow.start_run():
    # Log metrics
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log artifact
    mlflow.log_artifact(conf_matrix_path)
    
    # Log parameters
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("model_version", "Production")
    
    # Get Delta Table version
    silver_version = spark.sql(f"DESCRIBE HISTORY delta.`{SILVER_DELTA}`").collect()[0]["version"]
    mlflow.log_param("input_silver_table_version", silver_version)
    
print("Metrics and artifacts logged to MLflow.")

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# Read the Bronze Delta Table
gold_df_table = spark.read.format("delta").load(GOLD_DELTA)

# 1) How many mentions are there in the gold data total?
total_mentions = gold_df_table.count()
print(f"Total mentions in the gold data: {total_mentions}")

In [0]:
# 2) Count the number of neutral, positive, and negative tweets for each mention in new columns
mentions_df = (gold_df_table
    .groupBy("mention")
    .agg(
        count(when(col("sentiment") == 0, True)).alias("neutral_count"),
        count(when(col("sentiment") > 0, True)).alias("positive_count"),
        count(when(col("sentiment") < 0, True)).alias("negative_count")
    )
)

display(mentions_df)

In [0]:
# 3) Capture the total for each mention in a new column
mentions_df = mentions_df.withColumn("total_count", 
    col("neutral_count") + col("positive_count") + col("negative_count")
)

display(mentions_df)

In [0]:
# 4) Sort the mention count totals in descending order
sorted_mentions_df = mentions_df.orderBy(col("total_count").desc())

display(sorted_mentions_df)

In [0]:
# Convert to Pandas DataFrame for visualization
mentions_pd = sorted_mentions_df.toPandas()

# 5) Plot a bar chart of the top 20 mentions with positive sentiment
top_positive_mentions = mentions_pd.nlargest(20, 'positive_count')
fig_positive = px.bar(
    top_positive_mentions, 
    x='mention', 
    y='positive_count', 
    title='Top 20 Mentions with Positive Sentiment',
    labels={'mention': 'Mention', 'positive_count': 'Positive Count'},
    height=400
)

display(fig_positive)

In [0]:
# 6) Plot a bar chart of the top 20 mentions with negative sentiment
top_negative_mentions = mentions_pd.nlargest(20, 'negative_count')
fig_negative = px.bar(
    top_negative_mentions, 
    x='mention', 
    y='negative_count', 
    title='Top 20 Mentions with Negative Sentiment',
    labels={'mention': 'Mention', 'negative_count': 'Negative Count'},
    height=400
)

display(fig_negative)

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
import time
from pyspark.sql.streaming import StreamingQuery

# Start time of the notebook execution
start_time = time.time()

# Utility function to list all active streams
def list_active_streams():
    active_streams = StreamingQuery.active
    return active_streams

# List all active streams
active_streams = list_active_streams()

# Print active streams
print("Active Streams:")
for stream in active_streams:
    print(f"Stream Name: {stream.name}, Stream ID: {stream.id}")

# Stop all active streams
for stream in active_streams:
    print(f"Stopping Stream: {stream.name}")
    stream.stop()

# Calculate and print the elapsed time of the notebook
elapsed_time = time.time() - start_time
print(f"Elapsed time of the notebook: {elapsed_time:.2f} seconds")


In [0]:
# ENTER YOUR CODE HERE

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE